# train_v1

Uses the labeled subset `silver.word_states` data for training purposes

In [ ]:
%run './00_setup.ipynb'

In [ ]:
from src.trainutils import convert_df_to_features_v1, evaluate_thresholds
from src.envutils import is_databricks_env
from src.models.HybridFrequencyBinaryClassifier import HybridFrequencyBinaryClassifier
from pyspark.sql.types import ArrayType, DoubleType, FloatType
import pyspark.sql.functions as F

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
import joblib

In [ ]:
# Constants
C = 0.10
N_COMPONENTS = 300

In [ ]:
df = spark.sql("SELECT * FROM silver.word_states WHERE label is not NULL")

In [ ]:
df.count()

In [ ]:
# Convert frequencies to log frequencies
df = df.withColumn("log_frequency", F.log10(F.col("frequency") + 1))

# Concatenate log_frequency + features into single feature vector
final_df = convert_df_to_features_v1(df)

In [ ]:
# Convert to Pandas df to get X and y
pandas_df = final_df.toPandas()
X = pandas_df['features'].tolist()  # Convert Series of arrays to list of arrays
y = pandas_df['label'].tolist()
words = [row.word for row in df.select("word").collect()]
freq = np.array([row.log_frequency for row in df.select("log_frequency").collect()])

X = np.array(pandas_df['features'].tolist())  # Shape: (n_samples, 769)
y = np.array(pandas_df['label'].tolist())     # Shape: (n_samples,)

In [ ]:
# TODO: extract to utils file
def decompose_for_training(x_train, x_test, n_components=50):
    # Separate frequency and embeddings
    x_train_freq = x_train[:, :1]
    x_train_emb = x_train[:, 1:]

    x_test_freq = x_test[:, :1]
    x_test_emb = x_test[:, 1:]
    
    # Apply Truncated SVD to embeddings
    svd = TruncatedSVD(n_components=n_components, random_state=0)
    x_train_emb_reduced = svd.fit_transform(x_train_emb)
    x_test_emb_reduced = svd.transform(x_test_emb)

    # Save the svd decomposer to .joblib file
    with open(SVD_PATH, "wb") as f_svd:
        joblib.dump(svd, f_svd, protocol=None)

    # Recombine with frequency
    x_train_reduced = np.hstack([x_train_freq, x_train_emb_reduced])
    x_test_reduced = np.hstack([x_test_freq, x_test_emb_reduced])

    return x_train_reduced, x_test_reduced

In [ ]:
# Find optimized bin boundaries
# Choose 20 candidate thresholds and split into low and high ranges
quantiles = np.linspace(0.05, 0.95, 20)
candidates = np.quantile(freq, quantiles)

# Split into low and high ranges, avoiding complete overlap
low_range = candidates[:10]   # 5th to 50th percentile
high_range = candidates[10:]  # 50th to 95th percentile

thresholds, _ = evaluate_thresholds(freq, y, low_range, high_range)
low_threshold, high_threshold = thresholds

if low_threshold is None or high_threshold is None:
    # substitue hard coded values
    low_threshold = 1.68
    high_threshold = 4.72

In [ ]:
# Get training and testing set
SVD_PATH = './svd_v1.joblib' # TODO: Move this to constants with a path
x_train, x_test, y_train, y_test, words_train, words_test = train_test_split(
    X,
    y,
    words, 
    test_size=0.33, 
    random_state=0, 
    stratify=y
)

x_train_reduced, x_test_reduced = decompose_for_training(x_train, x_test, N_COMPONENTS)

In [ ]:
# Initialize model and train
low_freq_clf = GaussianNB()
mid_freq_clf = LogisticRegression(class_weight="balanced", max_iter=1000, C=C)
high_freq_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    class_weight='balanced',
    random_state=0
)
          
clf = HybridFrequencyBinaryClassifier(
    low_mid_threshold=low_threshold,
    mid_high_threshold=high_threshold,
    low_freq_model=low_freq_clf,
    mid_freq_model=mid_freq_clf,
    high_freq_model=high_freq_clf
)

clf.fit(x_train_reduced, y_train)

In [ ]:
y_pred = clf.predict(x_test_reduced)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_true=y_test, y_pred=y_pred)
y_proba = clf.predict_proba(x_test_reduced)
auc = roc_auc_score(y_true=y_test, y_score=y_proba[:, 1])

In [ ]:
print(f"confusion matrix:")
print(confusion_matrix(y_true=y_test, y_pred=y_pred))
print(f"accuracy: {acc}, f1: {f1}, AUC: {auc}")

In [ ]:
results = pd.DataFrame({
    "word": words_test,
    "frequency": x_test_reduced[:, 0],
    "true": y_test,
    "pred": y_pred
})

false_negatives = results[(results["true"] == 1) & (results["pred"] == 0)]
true_negatives = results[(results["true"] == 0) & (results["pred"] == 0)]
false_positives = results[(results["true"] == 0) & (results["pred"] == 1)]
true_positives = results[(results["true"] == 1) & (results["pred"] == 1)]
print(f"false negatives\n{false_negatives.head()}")
print(f"true negatives\n{true_negatives.head()}")
print(f"false positives\n{false_positives.head()}")
print(f"true positives\n{true_positives.head()}")

In [ ]:
# TODO: Create paths and filename constants for model and svd
# TODO: Save model
# Save model
# with open(H_MODEL_PATH, "wb") as f_model:
#     joblib.dump(clf, f_model, protocol=None)